In [209]:
from ipyleaflet import Map, DrawControl
import ee
import geemap
import datetime

In [305]:

# Initialize Earth Engine
ee.Initialize()

# Define Columbia Glacier AOI
columbia_glacier_center = ee.Geometry.Point([-147.1, 61.1])
aoi = columbia_glacier_center.buffer(15000)  # 5 km buffer

target_date = datetime.date(2009, 5, 17)  # Example target date
date_range = 15  # Search ±10 days around target date
start_date = (target_date - datetime.timedelta(days=date_range)).strftime('%Y-%m-%d')
end_date = (target_date + datetime.timedelta(days=date_range)).strftime('%Y-%m-%d')


datasets = {
    "ASTER": ("ASTER/AST_L1T_003", ['B3N', 'B02', 'B01'], {'min': 0, 'max': 0.3, 'bands': ['B3N', 'B02', 'B01']}),
    "Landsat 5": ('LANDSAT/LT05/C02/T1_TOA', ['B3', 'B2', 'B1'], {'min': 0, 'max': 0.2, 'bands': ['B3', 'B2', 'B1']}),
    "Landsat 7": ('LANDSAT/LE07/C02/T1_TOA', ['B3', 'B2', 'B1'], {'min': 0, 'max': 0.2, 'bands': ['B3', 'B2', 'B1']}),
    "Landsat 8": ("LANDSAT/LC08/C02/T1_TOA", ['B4', 'B3', 'B2'], {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}),
    "Landsat 9": ("LANDSAT/LC09/C02/T1", ['B4', 'B3', 'B2'], {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}),
    "Sentinel-1 SAR": ("COPERNICUS/S1_GRD", ['VV'], {'min': -30, 'max': -5, 'bands': ['VV']}),
    "Sentinel-2": ("COPERNICUS/S2", ['B4', 'B3', 'B2'], {'min': 0, 'max': 6000, 'bands': ['B4', 'B3', 'B2']})
}


In [306]:
print("Start Date:", start_date)
print("End Date:", end_date)


Start Date: 2009-05-02
End Date: 2009-06-01


In [307]:

# Find closest images
def get_closest_images(dataset, bands):
    collection = ee.ImageCollection(dataset).filterBounds(aoi).filterDate(start_date, end_date)
    sorted_images = collection.sort('system:time_start').toList(collection.size())
    return sorted_images, bands

image_list = []
for name, (dataset, bands, vis_params) in datasets.items():
    sorted_images, bands = get_closest_images(dataset, bands)
    try:
        count = sorted_images.size().getInfo()
    except:
        count=0
    if count > 0:
        image_list.append((name, sorted_images, bands, vis_params))
print(len(image_list)," images found")

2  images found


In [308]:
for i in range(len(image_list)):
    print(image_list[i])
    name, sorted_images, bands, vis_params = image_list[i]
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    print(date)

('Landsat 5', <ee.ee_list.List object at 0x16ae0f500>, ['B3', 'B2', 'B1'], {'min': 0, 'max': 0.2, 'bands': ['B3', 'B2', 'B1']})
2009-04-30
('Landsat 7', <ee.ee_list.List object at 0x16a0137d0>, ['B3', 'B2', 'B1'], {'min': 0, 'max': 0.2, 'bands': ['B3', 'B2', 'B1']})
2009-04-30


In [294]:
def update_map(index):
    name, sorted_images, bands, vis_params = image_list[index]
    image = ee.Image(sorted_images.get(0))
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    m.addLayer(image.select(bands), vis_params, f"{name} - {date}")
    print(f"Added {name} image from {date}.")
    return image,date



In [295]:
# Create interactive map
m = geemap.Map(center=[61.1, -147.1], zoom=10)
# Iteratively plot images
def plot_images():
    index = 0
    image,date=update_map(index)
    m.layout.width = '800px'
    m.layout.height = '600px'
    display(m)

    while index < len(image_list) - 1:
        response = input("Do you want to save this? (yes/no): ").strip().lower()
        if response != 'no':
            break
        index += 1
        image,date=update_map(index)
    return image,date

# Start the image plotting process
image,date=plot_images()

Added Landsat 5 image from 2009-05-01.


Map(center=[61.1, -147.1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

Do you want to save this? (yes/no):  no


Added Landsat 7 image from 2009-04-30.


In [282]:
geemap.ee_export_image(image, filename=f"/Users/sebinjohn/Downloads/tmp/tmp.tif", scale=40, region=aoi)
print(date)


Generating URL ...
Please wait ...
Data downloaded to /Users/sebinjohn/Downloads/tmp/tmp.tif
2005-05-05


In [296]:
# Define a list of coordinates for points
coordinates = [
    [-122.45, 37.75],  # Point 1
    [-122.48, 37.76],  # Point 2
    [-122.50, 37.77],  # Point 3
    # Add more points as needed
]

# Create a FeatureCollection of points
points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(coord), {'id': idx}) for idx, coord in enumerate(coordinates)
])

# Add the points to the map
m.addLayer(points, {'color': 'red'}, 'Points')

# Center the map on the points
m.centerObject(points, 10)

# Display the map
m


Map(bottom=74577.0, center=[37.7600017838033, -122.47666441367949], controls=(WidgetControl(options=['position…